In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import math
import time
from torch.utils.data import Dataset, DataLoader
from torch.cuda.amp import autocast, GradScaler
from pythainlp import word_tokenize, word_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🟢 Using device: {device}")

🟢 Using device: cuda


In [ ]:
MAX_LEN = 256
BATCH_SIZE = 128
EPOCHS = 500
LEARNING_RATE = 2e-4

In [4]:
df = pd.read_csv(r"D:\year4\สหกิจ\prachatai_train.csv")
texts = df["body_text"].astype(str).tolist()
label_cols = [
    "politics", "human_rights", "quality_of_life", "international",
    "social", "environment", "economics", "culture", "labor",
    "national_security", "ict", "education"
]
y = df[label_cols].values.astype(np.float32)

In [5]:
w2v = word_vector.WordVector(model_name="thai2fit_wv").get_model()
embedding_dim = w2v.vector_size
vocab_list = list(w2v.key_to_index.keys())

In [6]:
vocab = {"<PAD>": 0, "<UNK>": 1}
for i, word in enumerate(vocab_list, start=2):
    vocab[word] = i

In [7]:
tokenized_texts = [word_tokenize(t, keep_whitespace=False) for t in texts]

def encode_text(tokens, vocab):
    return [vocab.get(w, vocab["<UNK>"]) for w in tokens]

encoded_texts = [encode_text(tokens, vocab) for tokens in tokenized_texts]

In [ ]:
def pad_sequences(sequences, max_len=256, pad_value=0):
    padded = np.full((len(sequences), max_len), pad_value, dtype=np.int64)
    lengths = np.array([min(len(seq), max_len) for seq in sequences], dtype=np.int64) # กันค่าเกิน
    for i, seq in enumerate(sequences):
        end = min(len(seq), max_len)
        padded[i, :end] = seq[:end]
    return padded, lengths

X, lengths = pad_sequences(encoded_texts, max_len=MAX_LEN)

# First split: 80% train, 20% temp
X_train, X_temp, y_train, y_temp, len_train, len_temp = train_test_split(
    X, y, lengths, test_size=0.2, random_state=42
)
# Second split: 50% test, 50% valid (from temp = 10% each of total)
X_test, X_val, y_test, y_val, len_test, len_val = train_test_split(
    X_temp, y_temp, len_temp, test_size=0.5, random_state=42
)

In [ ]:
class ThaiTextDataset(Dataset):
    def __init__(self, X, lengths, y):
        self.X = torch.tensor(X, dtype=torch.long)
        self.lengths = torch.tensor(lengths, dtype=torch.long)
        self.y = torch.tensor(y, dtype=torch.float32)
    def __len__(self): return len(self.y)
    def __getitem__(self, idx): return self.X[idx], self.lengths[idx], self.y[idx]

train_loader = DataLoader(ThaiTextDataset(X_train, len_train, y_train), batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(ThaiTextDataset(X_test, len_test, y_test), batch_size=BATCH_SIZE)
val_loader = DataLoader(ThaiTextDataset(X_val, len_val, y_val), batch_size=BATCH_SIZE)

In [10]:
vocab_size = len(vocab)
embedding_matrix = np.zeros((vocab_size, embedding_dim))
for word, idx in vocab.items():
    if word in w2v:
        embedding_matrix[idx] = w2v[word]
    else:
        embedding_matrix[idx] = np.random.normal(scale=0.6, size=(embedding_dim,))

In [ ]:
class TransformerClassifier(nn.Module):
    def __init__(self, vocab_size, embed_dim, max_seq_len, output_dim, depth=4, heads=4, embedding_matrix=None):
        super(TransformerClassifier, self).__init__()
        self.embed_dim = embed_dim
        
        # 1. Embedding
        self.embedding = nn.Embedding(vocab_size, embed_dim, padding_idx=0)
        if embedding_matrix is not None:
            self.embedding.weight.data.copy_(torch.tensor(embedding_matrix, dtype=torch.float32))
            self.embedding.weight.requires_grad = True # Fine-tune embedding ด้วย
            
        # 2. Positional Encoding (จำเป็นสำหรับ Standard Transformer)
        self.pos_encoder = nn.Parameter(torch.randn(1, max_seq_len, embed_dim))
        
        # 3. Transformer Encoder (Native PyTorch = Fast)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=embed_dim, 
            nhead=heads, 
            dim_feedforward=embed_dim * 4,
            dropout=0.1,
            batch_first=True
        )
        self.transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=depth)
        
        # 4. Output
        self.fc = nn.Linear(embed_dim, output_dim)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x, lengths):
        # Create Padding Mask (Batch, Seq_Len) -> True where padding
        src_key_padding_mask = (x == 0)

        # Embed + Pos Encode
        x = self.embedding(x) * math.sqrt(self.embed_dim)
        x = x + self.pos_encoder[:, :x.size(1), :]
        
        # Transformer Pass
        x = self.transformer_encoder(x, src_key_padding_mask=src_key_padding_mask)
        
        # Mean Pooling (Ignore padding in calculation if possible, but basic mean is usually ok)
        x = x.mean(dim=1) 
        
        x = self.dropout(x)
        return self.fc(x)

In [ ]:
model = TransformerClassifier(
    vocab_size=vocab_size,
    embed_dim=embedding_dim,
    max_seq_len=MAX_LEN,
    output_dim=len(label_cols),
    depth=4,     # ไม่ต้องลึกมาก ข่าวไม่ได้ซับซ้อนขนาดนั้น
    heads=4,     # 300 หาร 6 ลงตัวพอดี
    embedding_matrix=embedding_matrix
).to(device)

In [ ]:
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.AdamW(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-4)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5)
scaler = GradScaler()

print(f"🚀 Start Training... (Max Epochs: {EPOCHS})")
best_loss = float('inf')

for epoch in range(EPOCHS):
    start_time = time.time()
    model.train()
    total_loss = 0
    
    for X_batch, lengths_batch, y_batch in train_loader:
        X_batch, lengths_batch, y_batch = X_batch.to(device), lengths_batch.to(device), y_batch.to(device)
        
        optimizer.zero_grad()
        
        with autocast(): # Mixed Precision
            outputs = model(X_batch, lengths_batch)
            loss = criterion(outputs, y_batch)
        
        scaler.scale(loss).backward()
        scaler.unscale_(optimizer)
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        scaler.step(optimizer)
        scaler.update()
        
        total_loss += loss.item()
    
    avg_loss = total_loss / len(train_loader)
    val_time = time.time() - start_time
    
    # Scheduler Update
    scheduler.step(avg_loss)
    current_lr = optimizer.param_groups[0]['lr']
    
    print(f"Epoch {epoch+1}/{EPOCHS} | Loss: {avg_loss:.4f} | LR: {current_lr:.6f} | Time: {val_time:.1f}s")

print("✅ Training Complete!")

c:\Users\nawapol\anaconda3\envs\PT\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🚀 Start Training... (Max Epochs: 500)


c:\Users\nawapol\anaconda3\envs\PT\Lib\site-packages\torch\nn\functional.py:5476: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:263.)
  attn_output = scaled_dot_product_attention(q, k, v, attn_mask, dropout_p, is_causal)


Epoch 1/500 | Loss: 0.2649 | LR: 0.000100 | Time: 93.7s
Epoch 2/500 | Loss: 0.1904 | LR: 0.000100 | Time: 95.8s
Epoch 3/500 | Loss: 0.1680 | LR: 0.000100 | Time: 97.0s
Epoch 4/500 | Loss: 0.1490 | LR: 0.000100 | Time: 97.3s
Epoch 5/500 | Loss: 0.1299 | LR: 0.000100 | Time: 96.8s
Epoch 6/500 | Loss: 0.1119 | LR: 0.000100 | Time: 98.1s
Epoch 7/500 | Loss: 0.0950 | LR: 0.000100 | Time: 98.2s
Epoch 8/500 | Loss: 0.0807 | LR: 0.000100 | Time: 98.4s
Epoch 9/500 | Loss: 0.0671 | LR: 0.000100 | Time: 98.7s
Epoch 10/500 | Loss: 0.0561 | LR: 0.000100 | Time: 98.4s
Epoch 11/500 | Loss: 0.0470 | LR: 0.000100 | Time: 98.3s
Epoch 12/500 | Loss: 0.0393 | LR: 0.000100 | Time: 102.2s
Epoch 13/500 | Loss: 0.0334 | LR: 0.000100 | Time: 98.6s
Epoch 14/500 | Loss: 0.0287 | LR: 0.000100 | Time: 114.4s
Epoch 15/500 | Loss: 0.0248 | LR: 0.000100 | Time: 113.3s
Epoch 16/500 | Loss: 0.0222 | LR: 0.000100 | Time: 113.8s
Epoch 17/500 | Loss: 0.0200 | LR: 0.000100 | Time: 121.1s
Epoch 18/500 | Loss: 0.0179 | LR: 0

In [14]:
model.load_state_dict(torch.load('best_model_thai2vec_transformer.pth'))
model.eval()

# F1 Score
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, lengths_batch, y_batch in test_loader:
        X_batch, lengths_batch = X_batch.to(device), lengths_batch.to(device)
        outputs = model(X_batch, lengths_batch)
        preds = torch.sigmoid(outputs).cpu().numpy()
        preds = (preds > 0.5).astype(int)
        y_true.append(y_batch.numpy())
        y_pred.append(preds)

y_true = np.vstack(y_true)
y_pred = np.vstack(y_pred)

print(f"\n🏆 F1-Macro: {f1_score(y_true, y_pred, average='macro'):.4f}")
print(f"🏆 F1-Micro: {f1_score(y_true, y_pred, average='micro'):.4f}")

c:\Users\nawapol\anaconda3\envs\PT\Lib\site-packages\torch\nn\modules\transformer.py:384: UserWarning: The PyTorch API of nested tensors is in prototype stage and will change in the near future. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\NestedTensorImpl.cpp:179.)
  output = torch._nested_tensor_from_mask(output, src_key_padding_mask.logical_not(), mask_check=False)



🏆 F1-Macro: 0.6202
🏆 F1-Micro: 0.6980


In [15]:
def predict(text):
    model.eval()
    tokens = word_tokenize(text, keep_whitespace=False)
    ids = [vocab.get(w, vocab["<UNK>"]) for w in tokens]
    
    # Handle Length
    if len(ids) > MAX_LEN: ids = ids[:MAX_LEN]
    else: ids = ids + [0] * (MAX_LEN - len(ids))
    
    tensor_in = torch.tensor([ids], dtype=torch.long).to(device)
    len_in = torch.tensor([min(len(tokens), MAX_LEN)], dtype=torch.long).to(device)
    
    with torch.no_grad():
        output = model(tensor_in, len_in)
        probs = torch.sigmoid(output).cpu().numpy()[0]
    
    results = []
    for i, prob in enumerate(probs):
        if prob > 0.5:
            results.append((label_cols[i], float(prob)))
            
    # ถ้าไม่มีอันไหนเกิน 0.5 เลย ให้เอาอันที่มากที่สุดมาอันนึง
    if not results:
        best_idx = np.argmax(probs)
        results.append((label_cols[best_idx], float(probs[best_idx])))
        
    return results

# Test Predict
print("\n🔮 Prediction Test:")
print(predict("รัฐบาลประกาศนโยบายการศึกษาใหม่ เพื่อพัฒนาเยาวชน"))


🔮 Prediction Test:
[('education', 0.6122887134552002)]


In [25]:
try:
    model.load_state_dict(torch.load('best_model_thai2vec_transformer.pth'))
    print("✅ Loaded Best Model successfully.")
except:
    print("⚠️ Warning: Could not load best model. Using current model state.")

model.eval()

# ---------------------------------------------------------
# B. วัดผลด้วยไฟล์ข้อสอบจริง (prachatai_test.csv)
# ---------------------------------------------------------
TEST_FILE_PATH = r"D:\year4\สหกิจ\prachatai_test.csv"  # 👈 แก้ Path ตรงนี้

try:
    print(f"\n📂 Loading Test File: {TEST_FILE_PATH} ...")
    df_test = pd.read_csv(TEST_FILE_PATH)
    
    # เตรียมข้อมูล
    test_texts = df_test["body_text"].astype(str).tolist()
    y_test_real = df_test[label_cols].values.astype(np.float32)

    # Tokenize & Encode
    tokenized_test = [word_tokenize(t, keep_whitespace=False) for t in test_texts]
    encoded_test = [encode_text(tokens, vocab) for tokens in tokenized_test]
    X_test_real, len_test_real = pad_sequences(encoded_test, max_len=MAX_LEN)

    # DataLoader
    real_test_loader = DataLoader(
        ThaiTextDataset(X_test_real, len_test_real, y_test_real), 
        batch_size=BATCH_SIZE, 
        shuffle=False
    )

    # Prediction Loop
    all_preds, all_labels = [], []
    with torch.no_grad():
        for X_batch, lengths_batch, y_batch in real_test_loader:
            X_batch, lengths_batch = X_batch.to(device), lengths_batch.to(device)
            outputs = model(X_batch, lengths_batch)
            preds = (torch.sigmoid(outputs).cpu().numpy() > 0.5).astype(int)
            all_preds.append(preds)
            all_labels.append(y_batch.numpy())

    all_preds = np.vstack(all_preds)
    all_labels = np.vstack(all_labels)

    # Report
    print("\n" + "="*35)
    print("🏆 FINAL TEST RESULTS (prachatai_test)")
    print("="*35)
    print(f"Macro F1-Score: {f1_score(all_labels, all_preds, average='macro'):.4f}")
    print(f"Micro F1-Score: {f1_score(all_labels, all_preds, average='micro'):.4f}")
    print("-" * 35)
    print(classification_report(all_labels, all_preds, target_names=label_cols, zero_division=0))

except Exception as e:
    print(f"\n❌ Skipped Test File Evaluation: {e}")
    print("(อาจเพราะหาไฟล์ไม่เจอ หรือชื่อ Column ไม่ตรง)")

# ---------------------------------------------------------
# C. ฟังก์ชันสำหรับลองพิมพ์เล่นเอง (Manual Predict)
# ---------------------------------------------------------
def predict(text, show_all=True, threshold=0.5):
    """
    show_all: ถ้า True จะแสดงคะแนนทุกหมวด (เรียงจากมากไปน้อย)
              ถ้า False จะแสดงเฉพาะหมวดที่เกิน threshold
    threshold: ค่าขั้นต่ำที่จะถือว่าเป็นหมวดที่ถูกต้อง (default = 0.5)
    """
    model.eval()
    tokens = word_tokenize(text, keep_whitespace=False)
    ids = [vocab.get(w, vocab["<UNK>"]) for w in tokens]
    
    if len(ids) > MAX_LEN: ids = ids[:MAX_LEN]
    else: ids = ids + [0] * (MAX_LEN - len(ids))
    
    tensor_in = torch.tensor([ids], dtype=torch.long).to(device)
    len_in = torch.tensor([min(len(tokens), MAX_LEN)], dtype=torch.long).to(device)
    
    with torch.no_grad():
        output = model(tensor_in, len_in)
        probs = torch.sigmoid(output).cpu().numpy()[0]
    
    # สร้าง list คู่ (ชื่อหมวด, คะแนน) ทุกหมวด
    all_results = [(label_cols[i], float(prob)) for i, prob in enumerate(probs)]
    # เรียงจากมากไปน้อย
    all_results.sort(key=lambda x: x[1], reverse=True)
    
    if show_all:
        return all_results
    else:
        # เฉพาะหมวดที่เกิน threshold
        filtered = [(name, prob) for name, prob in all_results if prob > threshold]
        if not filtered:
            # ถ้าไม่มีเลย เอาอันที่สูงสุดมาอันเดียว
            filtered = [all_results[0]]
        return filtered

✅ Loaded Best Model successfully.

📂 Loading Test File: D:\year4\สหกิจ\prachatai_test.csv ...

🏆 FINAL TEST RESULTS (prachatai_test)
Macro F1-Score: 0.6195
Micro F1-Score: 0.6914
-----------------------------------
                   precision    recall  f1-score   support

         politics       0.79      0.85      0.82      3842
     human_rights       0.66      0.60      0.63      1511
  quality_of_life       0.64      0.59      0.61      1127
    international       0.73      0.75      0.74       834
           social       0.41      0.29      0.34       789
      environment       0.79      0.79      0.79       772
        economics       0.60      0.52      0.56       519
          culture       0.56      0.48      0.52       398
            labor       0.79      0.75      0.77       350
national_security       0.55      0.49      0.52       338
              ict       0.68      0.65      0.66       292
        education       0.53      0.42      0.47       255

        micro av

In [ ]:
print("\n🔮 Manual Prediction Test:")
sample_text = "ประชาไท - มติพันธมิตรฯ ภาคใต้เดินหน้าจัดเวทีปฏิรูปการเมือง 29 - 30 เมษาฯ พร้อมลุยฟ้องศาลปกครองให้เพิกถอนการเลือกตั้ง 23 เมษาฯ ทนายชี้หลายเรื่องเอื้อประโยชน์ให้ไทยรักไทยเมื่อเวลา 10.30 น. วันที่ 9 เมษายน 2549 ที่ห้องประชุมอาคารบริหารคณะวิทยาการจัดการ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ มีการประชุมแกนนำพันธมิตรประชาชนเพื่อประชาธิปไตย ภาคใต้ มีผู้เข้าร่วม 30 คน โดยได้เชิญนายปกรณ์ รองสวัสดิ์ ประธานสภาทนายจังหวัดสงขลา มาให้ข้อมูลทางกฎหมายเกี่ยวกับการเลือกตั้งครั้งที่สอง ในวันที่ 23 เมษายน 2549นายปกรณ์ กล่าวในที่ประชุมว่า การดำเนินการของคณะกรรมการการเลือกตั้ง (กกต.) ในการประกาศรับสมัครเลือกตั้งสมาชิกสภาผู้แทนราษฎรครั้งที่ 2 ในเขตเลือกตั้งที่มีผู้สมัครได้คะแนนไม่ถึงร้อยละ 20 มีหลายอย่างที่เป็นการเอื้อให้กับพรรคไทยรักไทย และเป็นการทำผิดกฎหมาย เช่นการประกาศรับสมัครรับเลือกตั้งครั้งที่สองมีความผิดตามกฎหมายประกอประกอบรัฐธรรมนูญว่าด้วยการเลือกตั้งสมาชิกสภาผู้แทนราษฎรและสมาชิกวุฒิสภา มาตรา 74(2) ที่ระบุให้เลือกตั้งใหม่ ไม่ใช่รับสมัครเลือกตั้งใหม่ รวมทั้งการรับสมัครเลือกตั้งครั้งที่สองก็ไม่ได้มีการจับหมายเลข ทำให้พรรคไทยรักไทยที่ได้หมายเลข 2 ตั้งแต่การเลือกตั้งครั้งแรกจึงได้เปรียบเพราะได้หาเสียงไว้ก่อนแล้วนายปกรณ์ กล่าวอีกว่า สำหรับความผิดที่เกิดขึ้น ตามพระราชบัญญัติประกอบรัฐธรรมนูญว่าด้วยคณะกรรมการการเลือกตั้ง มาตรา 24 ระบุว่า ห้ามมิให้กรรมการการเลือกตั้ง กรรมการการเลือกตั้งในเขตเลือกตั้ง และอนุกรรมการการเลือกตั้ง กระทำการอันมิชอบที่ให้คุณให้โทษแก่ผู้สมัครหรือพรรคการเมือง ปฏิบัติหรือละเว้นการปฏิบัติโดยมิชอบ ส่วนการลงโทษนั้นตามมาตรา 29 ระบุว่า ให้จำคุก 1 -10 ปี ปรับ 20,000 - 200,000 บาท และขอให้ศาลเพิกถอนสิทธิทางการเมืองเป็นเวลา 10 ปี ที่ประชุมจึงลงมติว่าจะฟ้องร้องดำเนินคดีปกครองกับคณะกรรมการการเลือกตั้ง โดยจะฟ้องร้องต่อศาลปกครองสงขลา ขอให้ศาลปกครองมีคำสั่งเพิกถอนมติของคณะกรรมการการเลือกตั้งที่ให้เปิดสมัครรับเลือกตั้งครั้งที่สองในวันที่ 23 เมษายน 2549 และขอให้ยับยั้งการเลือกตั้งดังกล่าวด้วยนอกจากนี้ที่ประชุมยังมีมติจะเข้าร่วมปฏิรูปการเมือง ร่วมกับเครือข่ายองค์กรภาคประชาชนเพื่อปฏิรูปการเมืองและสังคม ส่วนการกรอบการดำเนินงานของพันธมิตรประชาชนเพื่อประชาธิปไตย ภาคใต้นั้น ที่ประชุมได้กำหนดจะจะจัดเวทีเรียนรู้เรื่องการปฏิรูปการเมืองในวันที่ 29 - 30 เมษายน 2549 โดยจะเชิญเจ้าของแนวคิดการปฏิรูปการเมืองฝ่ายต่างๆ มาให้ข้อมูลด้วย"
print(f"Input: {sample_text}")
print(f"Output: {predict(sample_text)}")


🔮 Manual Prediction Test:
Input: ประชาไท - มติพันธมิตรฯ ภาคใต้เดินหน้าจัดเวทีปฏิรูปการเมือง 29 - 30 เมษาฯ พร้อมลุยฟ้องศาลปกครองให้เพิกถอนการเลือกตั้ง 23 เมษาฯ ทนายชี้หลายเรื่องเอื้อประโยชน์ให้ไทยรักไทยเมื่อเวลา 10.30 น. วันที่ 9 เมษายน 2549 ที่ห้องประชุมอาคารบริหารคณะวิทยาการจัดการ มหาวิทยาลัยสงขลานครินทร์ วิทยาเขตหาดใหญ่ มีการประชุมแกนนำพันธมิตรประชาชนเพื่อประชาธิปไตย ภาคใต้ มีผู้เข้าร่วม 30 คน โดยได้เชิญนายปกรณ์ รองสวัสดิ์ ประธานสภาทนายจังหวัดสงขลา มาให้ข้อมูลทางกฎหมายเกี่ยวกับการเลือกตั้งครั้งที่สอง ในวันที่ 23 เมษายน 2549นายปกรณ์ กล่าวในที่ประชุมว่า การดำเนินการของคณะกรรมการการเลือกตั้ง (กกต.) ในการประกาศรับสมัครเลือกตั้งสมาชิกสภาผู้แทนราษฎรครั้งที่ 2 ในเขตเลือกตั้งที่มีผู้สมัครได้คะแนนไม่ถึงร้อยละ 20 มีหลายอย่างที่เป็นการเอื้อให้กับพรรคไทยรักไทย และเป็นการทำผิดกฎหมาย เช่นการประกาศรับสมัครรับเลือกตั้งครั้งที่สองมีความผิดตามกฎหมายประกอประกอบรัฐธรรมนูญว่าด้วยการเลือกตั้งสมาชิกสภาผู้แทนราษฎรและสมาชิกวุฒิสภา มาตรา 74(2) ที่ระบุให้เลือกตั้งใหม่ ไม่ใช่รับสมัครเลือกตั้งใหม่ รวมทั้งการรับ

In [ ]:
print("\n🔮 Manual Prediction Test:")
sample_text = "เมื่อวันที่ 1 ก.ย.ที่ผ่านมา ผู้จัดการออนไลน์ [1] รายงานว่า ที่สำนักงานศาลยุติธรรม ถนนรัชดาภิเษก เวลา 13.00 น. 1 ก.ย.59 พล.ต.นพ.เหรียญทอง แน่นหนา ผอ.รพ.มงกุฎวัฒนะ และแกนนำองค์กรเก็บขยะแผ่นดิน เดินทางมายื่นคำร้องถึงเลขาธิการสำนักงานศาลยุติธรรม คัดค้านการเสนอชื่อโปรดเกล้าฯ แต่งตั้งผู้ที่สอบผ่านการคัดเลือกเป็นผู้ช่วยผู้พิพากษารายหนึ่งที่เห็นว่ามีพฤติกรรมไม่เหมาะสมโดย พล.ต.นพ.เหรียญทอง ผอ.รพ.มงกุฎวัฒนะ กล่าวว่า การยื่นคัดค้านเนื่องจากเมื่อเดือน มิ.ย.ที่ผ่านมา ได้พบเห็นเพจเฟซบุ๊กของผู้ที่สอบผ่านการคัดเลือกเป็นผู้ช่วยผู้พิพากษารายนี้กดถูกใจข้อความที่มีบุคคลโพสต์ลักษณะหมิ่นสถาบันฯ ซึ่งตนได้นำข้อมูลชี้แจงให้ ป.ป.ช.ที่เป็นหน่วยงานต้นสังกัดของบุคคลนั้นทราบด้วย แต่บุคคลดังกล่าวได้ทำลายหลักฐานด้วยการปิดเพจเฟซบุ๊กนั้น แต่ตนสามารถเก็บข้อมูลการกระทำนั้นไว้ได้ทันวันนี้จึงนำมายื่นให้เลขาธิการสำนักงานศาลยุติธรรมไว้เป็นหลักฐานเพื่อให้เข้าสู่ที่ประชุมคณะกรรมการตุลาการศาลยุติธรรม (ก.ต.) พิจารณาตามขั้นตอนต่อไป เพราะตนเห็นว่าผู้ที่มีทัศนคติหมิ่นสถาบันฯ จะมาเป็นผู้พิพากษาได้อย่างไร"
print(f"Input: {sample_text}")
print(f"Output: {predict(sample_text)}")


🔮 Manual Prediction Test:
Input: เมื่อวันที่ 1 ก.ย.ที่ผ่านมา ผู้จัดการออนไลน์ [1] รายงานว่า ที่สำนักงานศาลยุติธรรม ถนนรัชดาภิเษก เวลา 13.00 น. 1 ก.ย.59 พล.ต.นพ.เหรียญทอง แน่นหนา ผอ.รพ.มงกุฎวัฒนะ และแกนนำองค์กรเก็บขยะแผ่นดิน เดินทางมายื่นคำร้องถึงเลขาธิการสำนักงานศาลยุติธรรม คัดค้านการเสนอชื่อโปรดเกล้าฯ แต่งตั้งผู้ที่สอบผ่านการคัดเลือกเป็นผู้ช่วยผู้พิพากษารายหนึ่งที่เห็นว่ามีพฤติกรรมไม่เหมาะสมโดย พล.ต.นพ.เหรียญทอง ผอ.รพ.มงกุฎวัฒนะ กล่าวว่า การยื่นคัดค้านเนื่องจากเมื่อเดือน มิ.ย.ที่ผ่านมา ได้พบเห็นเพจเฟซบุ๊กของผู้ที่สอบผ่านการคัดเลือกเป็นผู้ช่วยผู้พิพากษารายนี้กดถูกใจข้อความที่มีบุคคลโพสต์ลักษณะหมิ่นสถาบันฯ ซึ่งตนได้นำข้อมูลชี้แจงให้ ป.ป.ช.ที่เป็นหน่วยงานต้นสังกัดของบุคคลนั้นทราบด้วย แต่บุคคลดังกล่าวได้ทำลายหลักฐานด้วยการปิดเพจเฟซบุ๊กนั้น แต่ตนสามารถเก็บข้อมูลการกระทำนั้นไว้ได้ทันวันนี้จึงนำมายื่นให้เลขาธิการสำนักงานศาลยุติธรรมไว้เป็นหลักฐานเพื่อให้เข้าสู่ที่ประชุมคณะกรรมการตุลาการศาลยุติธรรม (ก.ต.) พิจารณาตามขั้นตอนต่อไป เพราะตนเห็นว่าผู้ที่มีทัศนคติหมิ่นสถาบันฯ จะมาเป็นผู้พิพากษาได้อย

In [28]:
print("\n🔮 Manual Prediction Test:")
sample_text = "17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์กับกลุ่มหัวรุนแรงหลังจากกลุ่ม IS ออกมาประกาศว่าเป็นผู้อยู่เบื้องหลังการโจมตีกรุงปารีสในคืนวันศุกร์ที่ผ่านมา ภาพในคลิปใน YouTube โฆษกของกลุ่มแฮคเกอร์สวมหน้ากากที่เป็นสัญลักษณ์ของกลุ่มได้ออกมาอ่านแถลงเป็นภาษาฝรั่งเศส มีใจความว่า จากการโจมตีของกลุ่ม IS ในกรุงปารีส กลุ่ม Anonymous ทั่วโลกจะตามล่ากลุ่ม IS เหมือนที่เคยทำตอนที่มีการโจมตีสำนักพิมพ์ Charlie Hebdo และครั้งนี้จะเป็นปฏิบัติการโจมตีครั้งใหญ่ที่สุดของกลุ่ม Anonymous เลย นอกจากนี้กลุ่ม Anonymous ยังแสดงความเสียใจต่อครอบครัวผู้สูญเสียในเหตุการณ์ครั้งนี้ กลุ่ม Anonymous เคยประกาศสงครามกับกลุ่ม IS หลังจากการโจมตีสำนักพิมพ์ Charlie Hebdo ที่ฝรั่งเศสเมื่อต้นปีที่ผ่านมา ซึ่งครั้งนั้นกลุ่ม Anonymous อ้างว่าได้ระงับบัญชีผู้ใช้งานที่เกี่ยวข้องกับ IS ไปหลายพันบัญชี (อ่านรายละเอียดเพิ่มเติม จากBlognone ที่กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์ขอกวาดล้างพวก ISIS [2])"
print(f"Input: {sample_text}")
print(f"Output: {predict(sample_text)}")


🔮 Manual Prediction Test:
Input: 17 พ.ย. 2558 Blognone [1] รายงานว่า กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์กับกลุ่มหัวรุนแรงหลังจากกลุ่ม IS ออกมาประกาศว่าเป็นผู้อยู่เบื้องหลังการโจมตีกรุงปารีสในคืนวันศุกร์ที่ผ่านมา ภาพในคลิปใน YouTube โฆษกของกลุ่มแฮคเกอร์สวมหน้ากากที่เป็นสัญลักษณ์ของกลุ่มได้ออกมาอ่านแถลงเป็นภาษาฝรั่งเศส มีใจความว่า จากการโจมตีของกลุ่ม IS ในกรุงปารีส กลุ่ม Anonymous ทั่วโลกจะตามล่ากลุ่ม IS เหมือนที่เคยทำตอนที่มีการโจมตีสำนักพิมพ์ Charlie Hebdo และครั้งนี้จะเป็นปฏิบัติการโจมตีครั้งใหญ่ที่สุดของกลุ่ม Anonymous เลย นอกจากนี้กลุ่ม Anonymous ยังแสดงความเสียใจต่อครอบครัวผู้สูญเสียในเหตุการณ์ครั้งนี้ กลุ่ม Anonymous เคยประกาศสงครามกับกลุ่ม IS หลังจากการโจมตีสำนักพิมพ์ Charlie Hebdo ที่ฝรั่งเศสเมื่อต้นปีที่ผ่านมา ซึ่งครั้งนั้นกลุ่ม Anonymous อ้างว่าได้ระงับบัญชีผู้ใช้งานที่เกี่ยวข้องกับ IS ไปหลายพันบัญชี (อ่านรายละเอียดเพิ่มเติม จากBlognone ที่กลุ่มแฮคเกอร์ Anonymous ประกาศสงครามไซเบอร์ขอกวาดล้างพวก ISIS [2])
Output: [('international', 0.9998990297317505), ('ict', 0.06944

In [29]:
print("\n🔮 Manual Prediction Test:")
sample_text = "กลุ่มนักกิจกรรมไซเบอร์ออกมาคัดค้านร่างกฎหมายความมั่นคงไซเบอร์ฉบับใหม่ที่ให้อำนาจเจ้าหน้าที่รัฐเข้าถึงข้อมูลแชทส่วนตัวของประชาชนได้โดยไม่ต้องขอหมายศาลโดยอ้างเหตุผลด้านความมั่นคง แต่ภาคประชาชนมองว่าเป็นการละเมิดสิทธิส่วนบุคคลและเสรีภาพในการแสดงออกบนโลกออนไลน์"
print(f"Input: {sample_text}")
print(f"Output: {predict(sample_text)}")


🔮 Manual Prediction Test:
Input: กลุ่มนักกิจกรรมไซเบอร์ออกมาคัดค้านร่างกฎหมายความมั่นคงไซเบอร์ฉบับใหม่ที่ให้อำนาจเจ้าหน้าที่รัฐเข้าถึงข้อมูลแชทส่วนตัวของประชาชนได้โดยไม่ต้องขอหมายศาลโดยอ้างเหตุผลด้านความมั่นคง แต่ภาคประชาชนมองว่าเป็นการละเมิดสิทธิส่วนบุคคลและเสรีภาพในการแสดงออกบนโลกออนไลน์
Output: [('politics', 0.9563563466072083), ('social', 0.7164331078529358), ('national_security', 0.6731414794921875), ('culture', 0.41415703296661377), ('ict', 0.34446677565574646), ('human_rights', 0.29110872745513916), ('environment', 0.1756698489189148), ('quality_of_life', 0.08780661970376968), ('labor', 0.06037697568535805), ('economics', 0.05850338190793991), ('international', 0.047269679605960846), ('education', 0.03665921092033386)]
